<a href="https://colab.research.google.com/github/Kabongosalomon/task-dataset-metric-extraction/blob/dataleakage/XLNet/XLNetScienceResultExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zihangdai/xlnet.git

Cloning into 'xlnet'...
remote: Enumerating objects: 122, done.
remote: Total 122 (delta 0), reused 0 (delta 0), pack-reused 122
Receiving objects: 100% (122/122), 2.92 MiB | 14.73 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 18kB/s 
     |████████████████████████████████| 3.8MB 38.0MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 512kB 42.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=9ccdb771d7c00f3025b9c9ba3fec0774727dfb42f01ceff2663c620659ba67c7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [3]:
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip

--2020-12-09 10:32:32--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.219.128, 108.177.112.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.219.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip’

cased_L-24_H-1024_A 100%[===================>]   1.25G  48.1MB/s    in 17s     

2020-12-09 10:32:49 (75.4 MB/s) - ‘cased_L-24_H-1024_A-16.zip’ saved [1338042341/1338042341]



In [4]:
!unzip cased_L-24_H-1024_A-16.zip

Archive:  cased_L-24_H-1024_A-16.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-24_H-1024_A-16/spiece.model  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_config.json  


In [5]:
import tensorflow as tf
tf.gfile.Rename('xlnet_cased_L-24_H-1024_A-16', 'xlnet_large_cased')

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# !python run_sci_classifier.py --use_tpu=True --tpu=grpc://10.68.220.162:8470 --do_train=True --do_eval=False --do_predict=False --task_name=sci --data_dir=/content/gdrive/My\ Drive/colab-ssh/few-shot-setup/NLP-TDMS/paperVersion --output_dir=$OUTPUT_DIR --model_dir=$OUTPUT_DIR --uncased=False --model_config_path=../xlnet_large_cased/xlnet_config.json --spiece_model_file=../xlnet_large_cased/spiece.model --init_checkpoint=$BUCKET_NAME/xlnet_large_cased/xlnet_model.ckpt --max_seq_length=512 --train_batch_size=16 --num_hosts=1 --num_core_per_host=8 --learning_rate=1e-5 --train_steps=500 --warmup_steps=500 --save_steps=100 --iterations=500

In [7]:
cp /content/gdrive/My\ Drive/colab-ssh/PythonCode/run_sci_classifier.py xlnet/

In [8]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 315.49 KiB | 4.10 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [9]:
cp bert/tokenization.py xlnet/

In [10]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.9MB/s 


In [11]:
cd xlnet/

/content/xlnet


In [12]:
import datetime
import json
import os
import pprint
import random
import string
import sys

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.96.187.122:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 24706152447455932),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17788076853646946246),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14846118731459752690),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 9606736164932596574),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 11543847113326076511),
 _DeviceAttributes(/job:tpu_worker/replica:0/tas

In [13]:
BUCKET = 'gs_colab_salomon' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'tdm_pwc_cased_xlnet16_fold2_train80_w600unkTAET_50000s_1000i' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://gs_colab_salomon/tdm_pwc_cased_xlnet16_fold2_train80_w600unkTAET_50000s_1000i *****


In [16]:
# !python run_sci_classifier.py --use_tpu=True --tpu=grpc://10.68.220.162:8470 --do_train=True --do_eval=False --do_predict=False --task_name=sci --data_dir=/content/gdrive/My\ Drive/colab-ssh/few-shot-setup/NLP-TDMS/paperVersion --output_dir=$OUTPUT_DIR --model_dir=$OUTPUT_DIR --uncased=False --model_config_path=../xlnet_large_cased/xlnet_config.json --spiece_model_file=../xlnet_large_cased/spiece.model --init_checkpoint=$BUCKET_NAME/xlnet_large_cased/xlnet_model.ckpt --max_seq_length=512 --train_batch_size=16 --num_hosts=1 --num_core_per_host=8 --learning_rate=1e-5 --train_steps=5 --warmup_steps=5 --save_steps=10 --iterations=50
# !python run_sci_classifier.py --use_tpu=True --tpu=grpc://10.68.220.162:8470 --do_train=True --do_eval=False --do_predict=False --task_name=sci --data_dir=/content/gdrive/My\ Drive/colab-ssh/paperswithcodedatawith600unk/twofold/fold2 --output_dir=$OUTPUT_DIR --model_dir=$OUTPUT_DIR --uncased=False --model_config_path=../xlnet_large_cased/xlnet_config.json --spiece_model_file=../xlnet_large_cased/spiece.model --init_checkpoint=$BUCKET_NAME/xlnet_large_cased/xlnet_model.ckpt --max_seq_length=512 --train_batch_size=16 --num_hosts=1 --num_core_per_host=8 --learning_rate=1e-5 --train_steps=50000 --warmup_steps=500 --save_steps=500 --iterations=1000

!python run_sci_classifier.py --use_tpu=True --tpu=grpc://10.68.220.162:8470 --do_train=False --do_eval=True --do_predict=False --task_name=sci --data_dir=/content/gdrive/My\ Drive/colab-ssh/paperswithcodedatawith600unk/twofold/fold2 --output_dir=$OUTPUT_DIR --model_dir=$OUTPUT_DIR --uncased=False --model_config_path=../xlnet_large_cased/xlnet_config.json --spiece_model_file=../xlnet_large_cased/spiece.model --init_checkpoint=$BUCKET_NAME/xlnet_large_cased/xlnet_model.ckpt --max_seq_length=512 --train_batch_size=16 --num_hosts=1 --num_core_per_host=8 --learning_rate=1e-5 --train_steps=50 --warmup_steps=10 --save_steps=10 --iterations=100




W1209 13:27:18.929487 139646143178624 module_wrapper.py:139] From run_sci_classifier.py:499: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1209 13:27:18.929743 139646143178624 module_wrapper.py:139] From run_sci_classifier.py:499: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1209 13:27:18.929957 139646143178624 module_wrapper.py:139] From run_sci_classifier.py:519: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1209 13:27:20.274788 139646143178624 lazy_loader.py:50] 
The TensorFlow contrib

In [ ]:
!python run_sci_classifier.py --use_tpu=True --tpu=grpc://10.61.49.210:8470 --do_train=False --do_eval=False --do_predict=True --eval_split=test --task_name=sci --data_dir=/content/gdrive/My\ Drive/paperswithcodedatawith600unk/twofold/fold2 --output_dir=$OUTPUT_DIR --model_dir=$OUTPUT_DIR --predict_dir=$OUTPUT_DIR --uncased=False --model_config_path=../xlnet_large_cased/xlnet_config.json --spiece_model_file=../xlnet_large_cased/spiece.model --init_checkpoint=$BUCKET_NAME/xlnet_large_cased/xlnet_model.ckpt --max_seq_length=512 --train_batch_size=16 --predict_batch_size=16 --num_hosts=1 --num_core_per_host=8 --learning_rate=1e-5 --train_steps=50000 --warmup_steps=500 --save_steps=500 --iterations=1000




W1207 16:22:40.109702 140067224254336 module_wrapper.py:139] From run_sci_classifier.py:499: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1207 16:22:40.109950 140067224254336 module_wrapper.py:139] From run_sci_classifier.py:499: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1207 16:22:40.110141 140067224254336 module_wrapper.py:139] From run_sci_classifier.py:507: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1207 16:22:41.383817 140067224254336 lazy_loader.py:50] 
The TensorFlow contrib